# Event log 
Notebook for reading data from eventLog

In [67]:
import numpy as np
import pandas as pd
import h5py
import ast

eventArg  = {}
eventName = {}
eventlog  = {}

path = '../datasets/'
key = '/events/'

fname = [
    'VRAcuityExp_2017-07-13_14-39-17_VR-4A_NIC',
    'VRAcuityExp_2017-07-13_15-05-16_VR-2B_NIC',
    'VRAcuityExp_2017-07-13_15-19-09_VR-2A_EDU',
    'VRAcuityExp_2017-07-13_15-38-34_VR-1A_NIC',
    'VRAcuityExp_2017-07-13_15-53-40_VR-1B_NIC',
    'VRAcuityExp_2017-07-13_16-11-46_VR-3A_NIC',
    'VRAcuityExp_2017-07-13_16-27-08_VR-3A_NIC',
    'VRAcuityExp_2017-07-13_17-09-07_VR-5A_NIC', ]


for i, x in enumerate(fname):
    eventlog[i] = pd.read_hdf(path+x+'.h5', key+'eventlog')      
    #eventArg[i] = h5py.File(path+x+'.h5', 'r')
    #eventName[i] = pd.read_hdf(path+x+h5, key+'eventNames')


In [95]:
#eventArg analysis
hdf5_file_name = path+fname[0]+'.h5'
dataset_name   = '/events/'

file    = h5py.File(hdf5_file_name, 'r')   # 'r' means that hdf5 file is open in read-only mode
dataset = file[dataset_name]

eventN = dataset['eventNames']
eventA = dataset['eventArguments']

for i, x in enumerate(eventA):
    print(type(x))
    print(x, '\n')
    event[i] = x
    

<class 'numpy.bytes_'>
b"{'var': <Mesh(name='Cylinder', position_rel=Translation(x=0.0, y=0.0, z=0.0), position_glob=(-0.19339873, 0.8927592, 0.14650956), rotation=RotationEulerDegrees(x=0.0, y=0.0, z=0.0)), 'attr': 'visible', 'value': False}" 

<class 'numpy.bytes_'>
b"{'duration': 60.0}" 

<class 'numpy.bytes_'>
b"{'var': <Mesh(name='Cylinder', position_rel=Translation(x=0.0, y=0.0, z=0.0), position_glob=(-0.19345596, 0.89346457, 0.14668623), rotation=RotationEulerDegrees(x=0.0, y=300.004699707, z=0.0)), 'attr': 'visible', 'value': True}" 

<class 'numpy.bytes_'>
b"{'var': <Mesh(name='Cylinder', position_rel=Translation(x=0.0, y=0.0, z=0.0), position_glob=(-0.19345497, 0.89346844, 0.14668664), rotation=RotationEulerDegrees(x=0.0, y=300.048248291, z=0.0)), 'attr': 'speed', 'value': 7.0}" 

<class 'numpy.bytes_'>
b"{'duration': 30.0}" 

<class 'numpy.bytes_'>
b"{'var': <Mesh(name='Cylinder', position_rel=Translation(x=0.0, y=0.0, z=0.0), position_glob=(-0.19344305, 0.89351153, 0.146695

In [107]:
for i, x in enumerate(eventA):
    event[i] = x

i=1
event[i] = event[i].decode('UTF-8')
print(event[i])
b = ast.literal_eval(event[i])
print(b)
print(b['duration'], '\n')
    
i=0
event[i] = event[i].decode('UTF-8')
print(event[i])
print(type(event[i]))
b = ast.literal_eval(event[i])
print(b)



{'duration': 60.0}
{'duration': 60.0}
60.0 

{'var': <Mesh(name='Cylinder', position_rel=Translation(x=0.0, y=0.0, z=0.0), position_glob=(-0.19339873, 0.8927592, 0.14650956), rotation=RotationEulerDegrees(x=0.0, y=0.0, z=0.0)), 'attr': 'visible', 'value': False}
<class 'str'>


SyntaxError: invalid syntax (<unknown>, line 1)

In [1]:
import json


print(json.loads('{"event": True}'))

for i, x in enumerate(eventA):
    event[i] = x

i=0
aa = event[i].decode('UTF-8')
print(type(aa))
print(aa)


a = json.loads("event[i]")
#a = json.loads(aa)

JSONDecodeError: Expecting value: line 1 column 11 (char 10)

In [2]:
import json
json_string = '{"favorited": false, "contributors": null}'
json_string = "{'favorited': false, 'contributors': null}"
value = json.loads(json_string)
print(value)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)